In [1]:
import matplotlib.pyplot as plt
from scipy.io import wavfile
import pandas as pd
import numpy as np
import os
import pathlib
import sys
import git

In [2]:
code_paths = {}
code_paths["repo_name"] = "p5-stress-classifier"

code_paths["repo_path"] = os.getcwd()
base_dir = os.path.basename(code_paths["repo_path"])
while base_dir != code_paths["repo_name"]:
    code_paths["repo_path"] = os.path.dirname(os.path.abspath(code_paths["repo_path"]))
    base_dir = os.path.basename(code_paths["repo_path"])

package_dir = pathlib.Path(code_paths["repo_path"], "src")
sys.path.append(str(package_dir))
from stresspred import (
    peak_time_to_rri,
    P5_StressDataLoader,
    timestamp_to_samp,
    resample_nonuniform,
    code_paths,
    timestamps_to_audacity_txt,
    find_files_with_string,
    hb_extract,
    frame_timestamps,
    find_local_hb_peaks,
    get_camel_case,
    write_dict_to_json,
    P5M5DataLoader
)

C:\Users\dbenesch\.conda\envs\stress-requirements\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [3]:
#find_files_with_string(string="pantompkins1985", file_dir=code_paths["repo_path"])

In [4]:
# things to experiment with
# changing highcut [20, 30]
# changing k_nearest_intervals [8, 12, 14]
# changing interpolation method to something more complex than linear, e.g. polynomial with order=3 (probably need to unpack dict for this **interpolation_args)
# or just argument with string
# can also try changing fixpeaks by height to be only the times that were changed
# can change size of HB template too...
# rri_anomalies thresholds
# threshold for peak fixing
# clean_method in fixpeaks_by_height
# time_boundaries in fixpeaks_by_height
# or just give option to pass clean_sig_info and raw_sig_info
# can adapt the time_boundaries in fix_peaks_by_height based on the heart rate variability of confident intervals
# or more outlier removal

In [6]:
version_id = 145
data_format = "8k"
sig_name = "ieml"

hb_extract_methods = ["temp", "pc", "critias_bp"]
repo = git.Repo(search_parent_directories=True)
git_hexsha = repo.head.object.hexsha
frame_len = 300
hb_extract_algo_kwargs = {
    "max_bpm": 200,
    "min_bpm": 40,
    "denoiser_type": "null",
    "thr_corr_height": -3,
    "min_n_confident_peaks": 20,
    "max_time_after_last_peak": 5,
    "clean_method": "own_filt",
    "highcut": 25,
    "relative_peak_height_for_temp_min": -2,
    "relative_peak_height_for_temp_max": 2,
    "temp_time_before_peak": 0.3,
    "temp_time_after_peak": 0.3,
    "fix_corr_peaks_by_height": False,
    "fix_interpl_peaks_by_height": False,
    "relative_rri_min": -2.5,
    "relative_rri_max": 2.5,
    "fixpeaks_by_height_time_boundaries": {
        "before_peak_clean": 0.1,
        "after_peak_clean": 0.1,
        "before_peak_raw": 0.005,
        "after_peak_raw": 0.005,
    },
    "corr_peak_extraction_method": "nk_ecg_process",
    "k_nearest_intervals": 8,
    "n_nan_estimation_method": "round",
    "interpolate_args": {"method": "spline", "order": 2},
    "output_format": "only_final",
    "debug_out_path": None,
}

In [7]:
hb_extract_algo_kwargs = {
    "max_bpm": 200,
    "min_bpm": 40,
    "denoiser_type": "null",
    "thr_corr_height": -3,
    "min_n_confident_peaks": 20,
    "max_time_after_last_peak": 5,
    "clean_method": "own_filt",
    "highcut": 25,
    "relative_peak_height_for_temp_min": -2,
    "relative_peak_height_for_temp_max": 2,
    "temp_time_before_peak": 0.3,
    "temp_time_after_peak": 0.3,
    "fix_corr_peaks_by_height": False,
    "fix_interpl_peaks_by_height": False,
    "relative_rri_min": -2.5,
    "relative_rri_max": 2.5,
    "fixpeaks_by_height_time_boundaries": {
        "before_peak_clean": 0.1,
        "after_peak_clean": 0.1,
        "before_peak_raw": 0.005,
        "after_peak_raw": 0.005,
    },
    "corr_peak_extraction_method": "nk_ecg_process",
    "k_nearest_intervals": 8,
    "n_nan_estimation_method": "round",
    #"interpolate_args": {"method": "spline", "order": 2},
    "interpolate_args": {"method": "akima"},
    "use_rri_to_peak_time": True,
    "move_average_rri_window": 3,
    "output_format": "only_final",
    "debug_out_path": None,
    }

In [11]:
loader = P5M5DataLoader()

all_sub_part_ids = loader.get_dataset_iterator(unsegmented=True)

In [12]:
all_sub_part_ids

[('P5M5_1', 'P01', '_unsegmented'),
 ('P5M5_1', 'P03', '_unsegmented'),
 ('P5M5_1', 'P04', '_unsegmented'),
 ('P5M5_1', 'P05', '_unsegmented'),
 ('P5M5_1', 'P07', '_unsegmented'),
 ('P5M5_1', 'P08', '_unsegmented'),
 ('P5M5_1', 'P09', '_unsegmented'),
 ('P5M5_1', 'P12', '_unsegmented'),
 ('P5M5_1', 'P14', '_unsegmented'),
 ('P5M5_1', 'P16', '_unsegmented'),
 ('P5M5_1', 'P17', '_unsegmented'),
 ('P5M5_1', 'P18', '_unsegmented'),
 ('P5M5_1', 'P24', '_unsegmented'),
 ('P5M5_1', 'P25', '_unsegmented'),
 ('P5M5_1', 'P26', '_unsegmented'),
 ('P5M5_2', 'P01', '_unsegmented'),
 ('P5M5_2', 'P02', '_unsegmented'),
 ('P5M5_2', 'P03', '_unsegmented'),
 ('P5M5_2', 'P04', '_unsegmented'),
 ('P5M5_2', 'P05', '_unsegmented'),
 ('P5M5_2', 'P06', '_unsegmented'),
 ('P5M5_2', 'P07', '_unsegmented'),
 ('P5M5_2', 'P08', '_unsegmented'),
 ('P5M5_2', 'P09', '_unsegmented'),
 ('P5M5_2', 'P10', '_unsegmented'),
 ('P5M5_2', 'P11', '_unsegmented'),
 ('P5M5_2', 'P12', '_unsegmented'),
 ('P5M5_2', 'P13', '_unsegme

In [13]:
root_out_path = pathlib.Path(loader.get_paths()["data_derivatives_dir"],"hb_annotations")


In [14]:
dataset_label, sub_id, cond_label = all_sub_part_ids[0]

In [15]:
sig_name = "ieml"

In [ ]:
for frame_len in [450, 600, 60, 90, 120]:
    for hb_extract_method in hb_extract_methods:
        version_id += 1
        for dataset_label, sub_id, cond_label in all_sub_part_ids:

            loader = P5M5DataLoader(dataset_label=dataset_label, sub_id=sub_id, cond_label=cond_label)

            sig_info = loader.get_sig(data_format="8k", sig_name=sig_name)

            if "nk" in hb_extract_method:
                auto_method_acronym = "NK"
            else:
                auto_method_acronym = hb_extract_method.upper()
            name_peaks_dict = {"zephyr_ecg": "R_Peak", "ti_ppg": "SP", "ieml": "S1_Peak"}
            label = name_peaks_dict[sig_info["name"]]


            txt_json_base_name = (
                dataset_label
                + "_"
                + sub_id
                + cond_label
                + "-"
                + sig_info["name"]
                + "-Ann-Auto-"
                + auto_method_acronym
                + "-"
                + name_peaks_dict[sig_info["name"]]
                + "_v"
                + str(version_id)
            )
            txt_file_name = txt_json_base_name + ".txt"
            json_file_name = txt_json_base_name + ".json"
            txt_path = str(
                pathlib.Path(
                    root_out_path,
                    dataset_label,
                    sub_id,
                    cond_label,
                    "v" + str(version_id),
                    txt_file_name,
                )
            )

            json_path = str(
                pathlib.Path(
                    root_out_path,
                    dataset_label,
                    sub_id,
                    cond_label,
                    "v" + str(version_id),
                    json_file_name,
                )
            )

            debug_out_path = str(
                pathlib.Path(
                    root_out_path,
                    dataset_label,
                    sub_id,
                    cond_label,
                    "v" + str(version_id),
                    txt_json_base_name + "_debug_out",
                )
            )


            hb_extract_params = hb_extract_algo_kwargs.copy()
            hb_extract_params["detector_type"] = hb_extract_method
            hb_extract_params["frame_len"] = frame_len
            hb_extract_params["validity"] = True
            hb_extract_params["version"] = version_id
            hb_extract_params["git_hexsha"] = git_hexsha

            write_dict_to_json(hb_extract_params, json_path=json_path)


            if not pathlib.Path(txt_path).is_file():
                sig_info["peak_time"] = frame_timestamps(
                    func=hb_extract,
                    sig=sig_info["sig"],
                    sig_time=sig_info["time"],
                    frame_len=frame_len,
                    sig_name=sig_info["name"],
                    method=hb_extract_method,
                    hb_extract_algo_kwargs=hb_extract_algo_kwargs,
                    save_file=True,
                    txt_path=txt_path,
                    label=label,
                )

C:\Users\dbenesch\eers\git\p5-stress-classifier\src\stresspred\preprocessing.py:479: UserWarning: Warning: the difference between timepoints is not uniform
  warn("Warning: the difference between timepoints is not uniform")
C:\Users\dbenesch\eers\git\p5-stress-classifier\src\stresspred\preprocessing.py:479: UserWarning: Warning: the difference between timepoints is not uniform
  warn("Warning: the difference between timepoints is not uniform")
C:\Users\dbenesch\eers\git\p5-stress-classifier\src\stresspred\preprocessing.py:479: UserWarning: Warning: the difference between timepoints is not uniform
  warn("Warning: the difference between timepoints is not uniform")
C:\Users\dbenesch\eers\git\p5-stress-classifier\src\stresspred\preprocessing.py:479: UserWarning: Warning: the difference between timepoints is not uniform
  warn("Warning: the difference between timepoints is not uniform")
C:\Users\dbenesch\eers\git\p5-stress-classifier\src\stresspred\preprocessing.py:479: UserWarning: Warnin

C:\Users\dbenesch\eers\git\p5-stress-classifier\src\stresspred\preprocessing.py:479: UserWarning: Warning: the difference between timepoints is not uniform
  warn("Warning: the difference between timepoints is not uniform")
C:\Users\dbenesch\AppData\Roaming\Python\Python39\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\dbenesch\AppData\Roaming\Python\Python39\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\dbenesch\eers\git\p5-stress-classifier\src\stresspred\preprocessing.py:479: UserWarning: Warning: the difference between timepoints is not uniform
  warn("Warning: the difference between timepoints is not uniform")
C:\Users\dbenesch\eers\git\p5-stress-classifier\src\stresspred\preprocessing.py:479: UserWarning: Warning: the difference between timepoints is not uniform
  warn("Warning: the di

In [13]:
dataset_label, sub_id, cond_label

('P5M5_1', 'P05', '_unsegmented')

In [8]:
all_sub_part_ids = P5_StressDataLoader().get_all_sub_part_ids()
all_sub_part_ids

[(1, 1), (2, 1), (3, 1), (4, 1), (4, 2), (5, 1), (6, 1)]

In [5]:
all_sub_part_ids = P5_StressDataLoader().get_all_sub_part_ids()
version_id = 158
data_format = "DB8k"
# sig_name = "zephyr_ecg"
sig_name = "ieml"
# hb_extract_method = "critias_bp"

# frame_len = 30
# frame_len = 180
# frame_len = 180
ecg_process_methods = [
    "neurokit",
    "pantompkins1985",
    "hamilton2002",
    "christov2004",
    "engzeemod2012",
    "elgendi2010",
    "kalidas2017",
]
hb_extract_method = "pc"
hb_extract_method = "nk_ppg_elgendi"
hb_extract_method = "temp"
repo = git.Repo(search_parent_directories=True)
git_hexsha = repo.head.object.hexsha
frame_len = 300



root_out_path = pathlib.Path("Z:\Shared\Documents\RD\RD2\_AudioRD\datasets\Biosignals\CritiasStress\data_derivatives\hb_annotations")
version_id += 1
for sub_id, part_id in all_sub_part_ids:

    loader = P5_StressDataLoader(sub_id=sub_id, part_id=part_id)
    sig_info = loader.get_sig(data_format=data_format, sig_name=sig_name)

    if "nk" in hb_extract_method:
        auto_method_acronym = "NK"
    else:
        auto_method_acronym = hb_extract_method.upper()
    name_peaks_dict = {"zephyr_ecg": "R_Peak", "ti_ppg": "SP", "ieml": "S1_Peak"}
    label = name_peaks_dict[sig_info["name"]]


    txt_json_base_name = (
        "P5_Stress-"
        + loader.sub_label
        + "_"
        + str(loader.part_id)
        + "-"
        + sig_info["name"]
        + "-Ann-Auto-"
        + auto_method_acronym
        + "-"
        + name_peaks_dict[sig_info["name"]]
        + "_v"
        + str(version_id)
    )
    txt_file_name = txt_json_base_name + ".txt"
    json_file_name = txt_json_base_name + ".json"
    txt_path = str(
        pathlib.Path(
            loader.set_paths(data_format="hb_annotations")["sub_data_dir"],
            "v" + str(version_id),
            txt_file_name,
        )
    )

    json_path = str(
        pathlib.Path(
            loader.set_paths(data_format="hb_annotations")["sub_data_dir"],
            "v" + str(version_id),
            json_file_name,
        )
    )

    debug_out_path = str(
        pathlib.Path(
            loader.set_paths(data_format="hb_annotations")["sub_data_dir"],
            "v" + str(version_id),
            txt_json_base_name + "_debug_out",
        )
    )

    hb_extract_algo_kwargs = {
    "max_bpm": 200,
    "min_bpm": 40,
    "denoiser_type": "null",
    "thr_corr_height": -3,
    "min_n_confident_peaks": 20,
    "max_time_after_last_peak": 5,
    "clean_method": "own_filt",
    "highcut": 25,
    "relative_peak_height_for_temp_min": -2,
    "relative_peak_height_for_temp_max": 2,
    "temp_time_before_peak": 0.3,
    "temp_time_after_peak": 0.3,
    "fix_corr_peaks_by_height": False,
    "fix_interpl_peaks_by_height": False,
    "relative_rri_min": -2.5,
    "relative_rri_max": 2.5,
    "fixpeaks_by_height_time_boundaries": {
        "before_peak_clean": 0.1,
        "after_peak_clean": 0.1,
        "before_peak_raw": 0.005,
        "after_peak_raw": 0.005,
    },
    "corr_peak_extraction_method": "nk_ecg_process",
    "k_nearest_intervals": 8,
    "n_nan_estimation_method": "round",
    #"interpolate_args": {"method": "spline", "order": 2},
    "move_average_rri_window": None,
    "interpolate_args": {"method": "akima"},
    "use_rri_to_peak_time": True,
    "find_anomalies_threshold": None,
    "output_format": "debug",
    "debug_out_path": debug_out_path,
    }
    hb_extract_params = hb_extract_algo_kwargs.copy()
    hb_extract_params["detector_type"] = hb_extract_method
    hb_extract_params["frame_len"] = frame_len
    hb_extract_params["validity"] = True
    hb_extract_params["version"] = version_id
    hb_extract_params["git_hexsha"] = git_hexsha

    write_dict_to_json(hb_extract_params, json_path=json_path)


    if not pathlib.Path(txt_path).is_file():
        sig_info["peak_time"] = frame_timestamps(
            func=hb_extract,
            sig=sig_info["sig"],
            sig_time=sig_info["time"],
            frame_len=frame_len,
            sig_name=sig_info["name"],
            method=hb_extract_method,
            hb_extract_algo_kwargs=hb_extract_algo_kwargs,
            save_file=True,
            txt_path=txt_path,
            label=label,
        )

C:\Users\dbenesch\AppData\Roaming\Python\Python39\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\dbenesch\AppData\Roaming\Python\Python39\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\dbenesch\AppData\Roaming\Python\Python39\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\dbenesch\AppData\Roaming\Python\Python39\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\dbenesch\AppData\Roaming\Python\Python39\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\dbenesch\AppData\Roaming\Python\Python39\site-packages\numpy\

In [11]:
all_sub_part_ids = P5_StressDataLoader().get_all_sub_part_ids()
version_id = 157
data_format = "DB8k"
# sig_name = "zephyr_ecg"
sig_name = "ieml"
# hb_extract_method = "critias_bp"

# frame_len = 30
# frame_len = 180
# frame_len = 180
ecg_process_methods = [
    "neurokit",
    "pantompkins1985",
    "hamilton2002",
    "christov2004",
    "engzeemod2012",
    "elgendi2010",
    "kalidas2017",
]
hb_extract_method = "pc"
hb_extract_method = "nk_ppg_elgendi"
hb_extract_method = "temp"
repo = git.Repo(search_parent_directories=True)
git_hexsha = repo.head.object.hexsha
frame_len = 300



root_out_path = pathlib.Path("Z:\Shared\Documents\RD\RD2\_AudioRD\datasets\Biosignals\CritiasStress\data_derivatives\hb_annotations")
version_id += 1
for sub_id, part_id in all_sub_part_ids:

    loader = P5_StressDataLoader(sub_id=sub_id, part_id=part_id)
    sig_info = loader.get_sig(data_format=data_format, sig_name=sig_name)

    if "nk" in hb_extract_method:
        auto_method_acronym = "NK"
    else:
        auto_method_acronym = hb_extract_method.upper()
    name_peaks_dict = {"zephyr_ecg": "R_Peak", "ti_ppg": "SP", "ieml": "S1_Peak"}
    label = name_peaks_dict[sig_info["name"]]


    txt_json_base_name = (
        "P5_Stress-"
        + loader.sub_label
        + "_"
        + str(loader.part_id)
        + "-"
        + sig_info["name"]
        + "-Ann-Auto-"
        + auto_method_acronym
        + "-"
        + name_peaks_dict[sig_info["name"]]
        + "_v"
        + str(version_id)
    )
    txt_file_name = txt_json_base_name + ".txt"
    json_file_name = txt_json_base_name + ".json"
    txt_path = str(
        pathlib.Path(
            loader.set_paths(data_format="hb_annotations")["sub_data_dir"],
            "v" + str(version_id),
            txt_file_name,
        )
    )

    json_path = str(
        pathlib.Path(
            loader.set_paths(data_format="hb_annotations")["sub_data_dir"],
            "v" + str(version_id),
            json_file_name,
        )
    )

    debug_out_path = str(
        pathlib.Path(
            loader.set_paths(data_format="hb_annotations")["sub_data_dir"],
            "v" + str(version_id),
            txt_json_base_name + "_debug_out",
        )
    )

    hb_extract_algo_kwargs = {
    "max_bpm": 200,
    "min_bpm": 40,
    "denoiser_type": "null",
    "thr_corr_height": -3,
    "min_n_confident_peaks": 20,
    "max_time_after_last_peak": 5,
    "clean_method": "own_filt",
    "highcut": 25,
    "relative_peak_height_for_temp_min": -2,
    "relative_peak_height_for_temp_max": 2,
    "temp_time_before_peak": 0.3,
    "temp_time_after_peak": 0.3,
    "fix_corr_peaks_by_height": False,
    "fix_interpl_peaks_by_height": False,
    "relative_rri_min": -2.5,
    "relative_rri_max": 2.5,
    "fixpeaks_by_height_time_boundaries": {
        "before_peak_clean": 0.1,
        "after_peak_clean": 0.1,
        "before_peak_raw": 0.005,
        "after_peak_raw": 0.005,
    },
    "corr_peak_extraction_method": "nk_ecg_process",
    "k_nearest_intervals": 8,
    "n_nan_estimation_method": "round",
    #"interpolate_args": {"method": "spline", "order": 2},
    "move_average_rri_window": None,
    "interpolate_args": {"method": "akima"},
    "use_rri_to_peak_time": True,
    "find_anomalies_threshold": None,
    "output_format": "only_final",
    "debug_out_path": None,
    }
    hb_extract_params = hb_extract_algo_kwargs.copy()
    hb_extract_params["detector_type"] = hb_extract_method
    hb_extract_params["frame_len"] = frame_len
    hb_extract_params["validity"] = True
    hb_extract_params["version"] = version_id
    hb_extract_params["git_hexsha"] = git_hexsha

    write_dict_to_json(hb_extract_params, json_path=json_path)


    if not pathlib.Path(txt_path).is_file():
        sig_info["peak_time"] = frame_timestamps(
            func=hb_extract,
            sig=sig_info["sig"],
            sig_time=sig_info["time"],
            frame_len=frame_len,
            sig_name=sig_info["name"],
            method=hb_extract_method,
            hb_extract_algo_kwargs=hb_extract_algo_kwargs,
            save_file=True,
            txt_path=txt_path,
            label=label,
        )

C:\Users\dbenesch\eers\git\p5-stress-classifier\src\stresspred\preprocessing.py:991: UserWarning: Warning: Z:Shared\Documents\RD\RD2\_AudioRD\datasets\Biosignals\CritiasStress\data_derivatives\hb_annotations\P01\v158\P5_Stress-P01_1-ieml-Ann-Auto-TEMP-S1_Peak_v158.json already exists.
  warn("Warning: " + json_path + " already exists.")
C:\Users\dbenesch\eers\git\p5-stress-classifier\src\stresspred\preprocessing.py:991: UserWarning: Warning: Z:Shared\Documents\RD\RD2\_AudioRD\datasets\Biosignals\CritiasStress\data_derivatives\hb_annotations\P02\v158\P5_Stress-P02_1-ieml-Ann-Auto-TEMP-S1_Peak_v158.json already exists.
  warn("Warning: " + json_path + " already exists.")
C:\Users\dbenesch\AppData\Roaming\Python\Python39\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\dbenesch\AppData\Roaming\Python\Python39\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in d

In [ ]:
loader.set_paths(data_format="hb_annotations")["sub_data_dir"]

In [ ]:
np.append(np.array([1, 2, 3]), 3)

In [ ]:
sig_info["peak_time"] = frame_timestamps(
    func=hb_extract,
    sig=sig_info["sig"],
    sig_time=sig_info["time"],
    frame_len=frame_len,
    sig_name=sig_info["name"],
    save_file=True,
    txt_path=txt_path,
    label=label,
)

In [11]:
all_sub_part_ids = P5_StressDataLoader().get_all_sub_part_ids()
for sub_id, part_id in all_sub_part_ids:

In [12]:
sig_info["peak_time"] = frame_timestamps(
    func=hb_extract,
    sig=sig_info["sig"],
    sig_time=sig_info["time"],
    frame_len=frame_len,
    sig_name=sig_info["name"],
    save_file=True,
    txt_path=txt_path,
    label=label,
)

In [6]:
timestamps_to_audacity_txt

<function stresspred.preprocessing.timestamps_to_audacity_txt(timestamp, txt_path='out.txt', label='timestamp', save=True, rewrite=False)>

In [ ]:
# part_id = 2
part_id = 1
# for sub_id in [4]:
sig_names = ["zephyr_ecg", "ti_ppg", "ieml"]
sig_names = ["zephyr_ecg", "ti_ppg"]
sig_names = ["ieml"]
for sub_id in [5]:
    for sig_name in sig_names:
        data_format = "DB8k"
        loader = P5_StressDataLoader(sub_id=sub_id, part_id=part_id)
        sig_info = loader.get_sig(data_format=data_format, sig_name=sig_name)
        data_format_paths = loader.get_paths(data_format=data_format)
        df_for_class = loader.get_timestamps_df_for_class()
        timestamps_to_audacity_txt(
            df_for_class.loc[:, ["start_time", "end_time"]].values,
            #str(data_format_paths["zephyr_ecg_sig"].stem) + "_task" + ".txt",
            ("P5_Stress-"
            + loader.sub_label
            + "_"
            + str(loader.part_id)
            + "-"
            + get_camel_case(sig_info["name"], first_upper=True)
            + "-Ann-Manual-LT-Task.txt"),
            label="to_label",
            rewrite=True
        )
        sig_info["peak_time"] = hb_extract(sig=sig_info["sig"], sig_time=sig_info["time"], sig_name=sig_info["name"])
        name_peaks_dict = {"zephyr_ecg": "R_Peak", "ti_ppg": "SP", "ieml": "S1_Peak"}
        if sig_name == "ieml":
            auto_method_acronym = "PC"
        else:
            auto_method_acronym = "NK"
        timestamps_to_audacity_txt(
            sig_info["peak_time"],
            # str(data_format_paths[sig_info["name"] + "_sig"].stem) + "_peaks_auto" + ".txt",
            "P5_Stress-"
            + loader.sub_label
            + "_"
            + str(loader.part_id)
            + "-"
            + get_camel_case(sig_info["name"], first_upper=True)
            + "-Ann-Auto-"
            + auto_method_acronym
            + "-"
            + name_peaks_dict[sig_info["name"]]
            + ".txt",
            label=name_peaks_dict[sig_info["name"]],
            rewrite=True
        )

In [4]:
# part_id = 2
part_id = 1
# for sub_id in [4]:
sig_names = ["zephyr_ecg", "ti_ppg", "ieml"]
sig_names = ["ieml"]
sig_names = ["zephyr_ecg", "ti_ppg"]
for sub_id in [5]:
    for sig_name in sig_names:
        data_format = "DB8k"
        loader = P5_StressDataLoader(sub_id=sub_id, part_id=part_id)
        sig_info = loader.get_sig(data_format=data_format, sig_name=sig_name)
        data_format_paths = loader.get_paths(data_format=data_format)
        df_for_class = loader.get_timestamps_df_for_class()
        timestamps_to_audacity_txt(
            df_for_class.loc[:, ["start_time", "end_time"]].values,
            #str(data_format_paths["zephyr_ecg_sig"].stem) + "_task" + ".txt",
            ("P5_Stress-"
            + loader.sub_label
            + "_"
            + str(loader.part_id)
            + "-"
            + get_camel_case(sig_info["name"], first_upper=True)
            + "-Ann-Manual-LT-Task.txt"),
            label="to_label",
            rewrite=True
        )
        sig_info["peak_time"] = hb_extract(sig=sig_info["sig"], sig_time=sig_info["time"], sig_name=sig_info["name"])
        name_peaks_dict = {"zephyr_ecg": "R_Peak", "ti_ppg": "SP", "ieml": "S1_Peak"}
        timestamps_to_audacity_txt(
            sig_info["peak_time"],
            # str(data_format_paths[sig_info["name"] + "_sig"].stem) + "_peaks_auto" + ".txt",
            "P5_Stress-"
            + loader.sub_label
            + "_"
            + str(loader.part_id)
            + "-"
            + get_camel_case(sig_info["name"], first_upper=True)
            + "-Ann-Auto-NK-"
            + name_peaks_dict[sig_info["name"]]
            + ".txt",
            label=name_peaks_dict[sig_info["name"]],
            rewrite=True
        )

{'label': ['rest_task_mentalNoise', 'rest_task_mental', 'rest_task_cpt', 'rest_task_speechBaseline', 'rest_task_speechStressed', 'task_mentalNoise', 'task_mental', 'task_cpt', 'task_speechBaseline', 'task_speechStressed'], 'start_time': array([ 215.84375925,  873.25803475, 1537.942431  , 2206.231749  ,
       2879.155857  ,  508.9490462 , 1176.175937  , 1834.999488  ,
       2525.009788  , 3170.219649  ]), 'end_time': array([ 395.84375925, 1053.25803475, 1717.942431  , 2386.231749  ,
       3059.155857  ,  688.9490462 , 1356.175937  , 2014.999488  ,
       2705.009788  , 3350.219649  ])}
{'label': ['rest_task_mentalNoise', 'rest_task_mental', 'rest_task_cpt', 'rest_task_speechBaseline', 'rest_task_speechStressed', 'task_mentalNoise', 'task_mental', 'task_cpt', 'task_speechBaseline', 'task_speechStressed'], 'start_time': array([ 215.84375925,  873.25803475, 1537.942431  , 2206.231749  ,
       2879.155857  ,  508.9490462 , 1176.175937  , 1834.999488  ,
       2525.009788  , 3170.219649 

In [5]:
sig_info["peak_time"]

array([8.71891149e-01, 1.32683435e+00, 2.07174135e+00, ...,
       3.55360135e+03, 3.55471021e+03, 3.55554211e+03])

In [8]:
type(hb_extract)

NoneType

In [ ]:
timestamps_to_audacity_txt

In [ ]:
"""
sig_info["cor_peak_time"] = find_local_hb_peaks(sig_info["peak_time"], 
                     sig=sig_info["sig"], 
                     sig_time=sig_info["time"], 
                     sampling_rate=sig_info["sampling_rate"], 
                     check_height_outlier=True)
"""

In [ ]:
# should be two outputs of TEMP: peaks we are confident in, and complete interpolated peak times
# parameters I should try changing: 

In [ ]:
s = "ti_ppg"
s = re.sub(r"(_|-)+", " ", s).title().replace(" ", "")
s

In [ ]:
P5_Stress-P01-TiPpg-Ann-Manual-LT-Task

In [ ]:
P5_Stress-P01-TiPpg-Ann-Auto-NK-SP